In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, VBox, HBox, Output
from IPython.display import display, Markdown
from google.colab import files



# -----------------------------
# Last opp fil
# -----------------------------
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Pandas – automatisk separator, desimal=komma
df = pd.read_csv(filename, sep=None, engine='python', decimal=',')

tid_data = df.iloc[:,0].values
niva_data = df.iloc[:,1].values

# -----------------------------
# Automatisk estimering
# -----------------------------
y0_est = niva_data[0]
A_est = niva_data[-1] - y0_est
thresh10= y0_est + 0.1 * A_est
thresh85 = y0_est + 0.85 * A_est

L_est10 = tid_data[np.where(niva_data > thresh10)[0][0]]
L_est85 = tid_data[np.where(niva_data > thresh85)[0][0]]
L_est = abs(L_est10-0.13*(L_est85-L_est10))
thresh63 = y0_est + 0.63 * A_est
T_est = tid_data[np.where(niva_data > thresh63)[0][0]]

display(Markdown(
    f"**Autoestimat:** A={A_est:.2f}, $T_{{63\\%}}$={T_est:.2f}, L={L_est:.2f}, y0={y0_est:.2f}"
))

# -----------------------------
# Plott for visning
# -----------------------------
def plot_fopdt_display(A, T, L, y0):
    y_model = np.where(
        tid_data < L,
        y0,
        y0 + A * (1 - np.exp(-(tid_data - L) / T))
    )

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(tid_data, niva_data, "b.", markersize=3, label="Måledata")
    ax.plot(tid_data, y_model, "r-",
            label=f"Modell: A={A:.2f}, $T_{{63\\%}}$={T:.2f}, L={L:.2f}, y0={y0:.2f}")
    #hjelpeliner:
    #y0 line:
    ax.plot([0,L],[y0,y0])
    #vertikal hjelpelinje for y0:
    ax.plot([47,47],[0,y0],linestyle='dotted')
    #vertikal hjelpeline for 63%
    ax.plot([T,T],[y0,y0+A],linestyle='dotted')
    #hjelpelinje for A
    ax.plot([tid_data[-1],tid_data[-1]],[y0,y0+A * (1 - np.exp(-(tid_data[-1] - L) / T))],linestyle='dotted')
    #hjelpelinje for L
    ax.plot([L,L],[0,A*0.5],linestyle='dotted')
    #horisontal hjelpelinje for 63%
    ax.plot([0,tid_data[-1]],[y0+A *(1 - np.exp(-(T - L) / T)),y0+A*(1 - np.exp(-(T - L) / T))],linestyle='dotted')
    #hjelpelinje for y0
    ax.plot([0,tid_data[-1]],[y0,y0],linestyle='dotted')
    #labels for hjelpelinjer
    ax.text(T+4,0.63*A/2,f"$A_{{63\\%}}$={A*0.63:.2f}")
    ax.text(tid_data[-1]-35,A/2,f"A={A:.2f}")
    ax.text(L-8,y0+8,f"L={L:.2f}",rotation=90)
    ax.text(L+10,y0+A*(1 - np.exp(-(T - L) / T)),f"$T_{{63\\%}}$={T:.2f}")
    #aksetitler
    ax.set_xlabel("Tid [t]")
    ax.set_ylabel("Nivå / respons")
    ax.grid(True)
    ax.legend()

    plt.show()

# -----------------------------
# Plott kun for PNG
# -----------------------------
def plot_fopdt_png(A, T, L, y0):
    y_model = np.where(
        tid_data < L,
        y0,
        y0 + A * (1 - np.exp(-(tid_data - L) / T))
    )

    fig, ax = plt.subplots(figsize=(11.69, 8.27))
    ax.plot(tid_data, niva_data, "b.", markersize=3, label="Måledata")
    ax.plot(tid_data, y_model, "r-",
            label=f"Modell: A={A:.2f}, $T_{{63\\%}}$={T:.2f}, L={L:.2f}, y0={y0:.2f}")
    #hjelpeliner:
    #y0 line:
    ax.plot([0,L],[y0,y0])
    #vertikal hjelpelinje for y0:
    ax.plot([47,47],[0,y0],linestyle='dotted')
    #vertikal hjelpeline for 63%
    ax.plot([T,T],[y0,y0+A],linestyle='dotted')
    #hjelpelinje for A
    ax.plot([tid_data[-1],tid_data[-1]],[y0,y0+A * (1 - np.exp(-(tid_data[-1] - L) / T))],linestyle='dotted')
    #hjelpelinje for L
    ax.plot([L,L],[0,A*0.5],linestyle='dotted')
    #horisontal hjelpelinje for 63%
    ax.plot([0,tid_data[-1]],[y0+A *(1 - np.exp(-(T - L) / T)),y0+A*(1 - np.exp(-(T - L) / T))],linestyle='dotted')
    #hjelpelinje for y0
    ax.plot([0,tid_data[-1]],[y0,y0],linestyle='dotted')
    #labels for hjelpelinjer
    ax.text(T+4,0.63*A/2,f"$A_{{63\\%}}$={A*0.63:.2f}")
    ax.text(tid_data[-1]-35,A/2,f"A={A:.2f}")
    ax.text(L-8,y0+8,f"L={L:.2f}",rotation=90)
    ax.text(L+10,y0+A*(1 - np.exp(-(T - L) / T)),f"$T_{{63\\%}}$={T:.2f}")
    ax.text(50,y0-4,f"$y_0$={y0:.2f}")
    ax.set_xlabel("Tid [t]")
    ax.set_ylabel("Nivå / respons")
    ax.grid(True)
    ax.legend()

    filepath = "FOPDT_plot.png"
    fig.canvas.draw()
    fig.savefig(filepath, dpi=600, bbox_inches="tight")
    plt.close(fig)
    files.download(filepath)

# -----------------------------
# Widgets
# -----------------------------
A_slider = FloatSlider(value=A_est, min=0, max=2*A_est, step=0.1, description="A")
T_slider = FloatSlider(value=T_est, min=1, max=2*T_est, step=0.1, description="T₆₃\uFE6A")
L_slider = FloatSlider(value=L_est, min=0, max=8*L_est, step=0.1, description="L")
y0_slider = FloatSlider(value=y0_est, min=0, max=2*y0_est, step=0.1, description="y0")

save_button = Button(description="Last ned PNG")

out = Output()

# -----------------------------
# Oppdater plott når slider endres
# -----------------------------
def update_plot(change):
    with out:
        out.clear_output(wait=True)
        plot_fopdt_display(A_slider.value, T_slider.value,
                           L_slider.value, y0_slider.value)

A_slider.observe(update_plot, "value")
T_slider.observe(update_plot, "value")
L_slider.observe(update_plot, "value")
y0_slider.observe(update_plot, "value")

save_button.on_click(lambda b: plot_fopdt_png(
    A_slider.value, T_slider.value, L_slider.value, y0_slider.value
))

# Layout
ui = HBox([out, VBox([A_slider, T_slider, L_slider, y0_slider, save_button])])
display(ui)

# Første plot
update_plot(None)

# Instruksjoner
display(Markdown(
r"""
## Instruksjoner til elevene

1. Dra sliderne for A, T, L og y0 for å tilpasse modellen til måledataene.
2. Den røde kurven (modellen) oppdateres automatisk hver gang du endrer verdiene.
3. Trykk **Last ned PNG** for å lagre plottet.
4. Les av verdiene A, T, L og y0 fra grafen.
5. Bruk dem til å fylle inn PID-formlene:

- $K = \frac{A}{\Delta u}$

- $K_p = \frac{T}{K(\lambda + L)}$
- $T_i = T$
- $T_d = \frac{L \cdot T}{L+\lambda}$
- $\lambda = L$

6. Diskuter hvordan valg av \(\lambda\) påvirker responsen.
"""
))


Saving pid_test.txt to pid_test (62).txt


**Autoestimat:** A=77.20, $T_{63\%}$=97.00, L=1.37, y0=5.30


## Instruksjoner til elevene

1. Dra sliderne for A, T, L og y0 for å tilpasse modellen til måledataene.
2. Den røde kurven (modellen) oppdateres automatisk hver gang du endrer verdiene.
3. Trykk **Last ned PNG** for å lagre plottet.
4. Les av verdiene A, T, L og y0 fra grafen.
5. Bruk dem til å fylle inn PID-formlene:

- $K = \frac{A}{\Delta u}$

- $K_p = \frac{T}{K(\lambda + L)}$
- $T_i = T$
- $T_d = \frac{L \cdot T}{L+\lambda}$
- $\lambda = L$

6. Diskuter hvordan valg av \(\lambda\) påvirker responsen.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# FOPDT-modellering – Interaktiv Colab-notatbok

Last opp en datafil (`pid_test.txt`), juster parametre med sliders,
og last ned ferdig figur som PNG.

In [ ]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()